In [ ]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.ibmq import RunnerResult
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Statevector

# Loading your IBM Quantum account(s)


provider = IBMQ.get_provider(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')
backend_use = provider.get_backend('ibmq_qasm_simulator')

k = 5
state = (1/np.sqrt(8))*np.array([np.exp(-1j*2*np.pi*k*(0)/8),np.exp(-1j*2*np.pi*k*(1)/8),
                                 np.exp(-1j*2*np.pi*k*(2)/8),
 np.exp(-1j*2*np.pi*k*3/8),np.exp(-1j*2*np.pi*k*4/8),np.exp(-1j*2*np.pi*k*5/8),
 np.exp(-1j*2*np.pi*k*6/8), np.exp(-1j*2*np.pi*k*7/8)])

qubits = 3

circuit = QuantumCircuit(qubits, qubits)
circuit.initialize(state)
circuit.append(QFT(qubits), range(qubits))
circuit.measure(range(qubits), range(qubits))

N = 2
epsilon = 0.5
mass = 1.0

#define gauge_kinetic to take in only epsilon
def gauge_kinetic(epsilon):
    #create a circuit, since this gate only ever works on 1 qubit, your circuit need only 1 qubit
    circuit=QuantumCircuit(1)
    #apply the correct gate to your circuit
    circuit.rx(-epsilon/2,0)
    #turn the circuit into a gate, U_kg
    U_kg = circuit.to_gate()
    #Name your gate something useful for looking at the circuits later
    U_kg.name = "U$_{Kg}$"
    #return your gate
    return U_kg

def fermion_mass(epsilon,mass,eta):
    circuit=QuantumCircuit(1)
    circuit.rz(-epsilon*mass * eta,0)
    U_m = circuit.to_gate()
    U_m.name = "U$_m$"
    return U_m

def fermion_hopping_opt2(epsilon,eta):
    circuit= QuantumCircuit(3)
    circuit.cx(0,2)
    circuit.h(0)
    circuit.cx(1,0)
    circuit.cx(0,2)
    circuit.rz(epsilon/4 * eta,0)
    circuit.rz(-epsilon/4 * eta,2)
    circuit.cx(0,2)
    circuit.cx(1,0)
    circuit.h(0)
    circuit.cx(0,2)
    U_fho2 = circuit.to_gate()
    U_fho2.name = "U$_{fho2}$"
    return U_fho2

#define your function to take in a dictionary counts
def get_mean_fermion_number(counts):
    #first define a variable mean and set to 0
    mean = 0
    #define a variable total_counts = sum(counts.values())
    total_counts = sum(counts.values())
    #perform a for loop over the elements in counts, using for example `s` as the loop variable
    for s in counts:
        #define a variable p which is the last element of the identifier in your loop variable
        #i.e. p = s[-1]
        p = s[-1]
        #check if p is a `1`.
        if p == '1':
            #if p == '1', then add to mean counts[s]/total_counts
            mean += 1./total_counts * counts[s]
    return mean

#define a function `get_bootstrap_error(counts)` which takes in a dictionary `counts`
def get_bootstrap_error(counts):
    #compute the nshots by using sum(counts.values())
    nshots=sum(counts.values())
    #For our statistics, we will resample the distribution 100 times, so set B=100
    B = 100
    #Extract the list of unique keys in your dictionary and store them as a variable k via
    # k = list(counts.keys())
    k = list(counts.keys())
    #For each key, we need to define a probability list prob via [counts[a]/nshots for a in k]
    prob = [counts[a]/nshots for a in k]
    #define an empty array means
    means = []
    #With all this setup, now perform a for loop over b in range(B)
    for b in range(B):
        #set a variable m=0
        m = 0
        #build a new set of samples using numpy.random.choice with the arguments k, size=nshots, and p=prob
        #this function will return nshots worth of new results with probability prob from the choices in k
        samples = np.random.choice(k, size=nshots, p=prob)
        #make a for loop over s in the list samples
        for s in samples:
            #set a new variable p = s[-1] to again extract the number of electrons
            p = s[-1]
            #check if p = `1`
            if p == '1':
                #add 1/nshots to m if true
                m += 1./nshots
        #append the results of m to means
        means.append(m)
    #return from the function the standard deviation of means via numpy.std(means)   
    return np.std(means)

counts=[]
sim_counts=[]
means=[]
errs=[]
sim_means=[]
sim_errs=[]
Ts=[]
noisy_counts = []
noisy_means = []

#Make the for loop
for T in range(int(0/epsilon),int(3/epsilon)):
    #Append Ts and create your circuit
    Ts.append(T)
    qc = QuantumCircuit(2*N-1, 2*N-1)

    #state prep!
    qc.x(0)
    qc.h(0)
    


    #time evolution
    for t in range(T):
        for n in range(0,2*N,2):
            qc.append(fermion_mass(epsilon,mass,(-1)**(n/2+1)),[n])
        for l in range(1,2*N-1,2):
            qc.append(gauge_kinetic(epsilon),[l])
        for n in range(0,2*N-3,2):
            qc.append(fermion_hopping_opt2(epsilon, (-1)**(n/2)),[n,n+1,n+2])
      
    #add measurement steps
    qc.measure(range(3), range(3))

    
    #define your two backends
    backend = provider.backend.ibmq_qasm_simulator
    #backend = provider.get_backend('ibm_lagos')
    
    #transpile your circuit for each backend
    sim_tc = transpile(qc, sim_backend, optimization_level=3)
    #tc = transpile(qc, backend, optimization_level=3)
    program_inputs = {
        'circuits': circuit,
        'optimization_level': 3,
        'measurement_error_mitigation': True
    }
    options = {'backend_name': backend.name()}
    job = provider.runtime.run(program_id="circuit-runner",
                               options=options,
                               inputs=program_inputs,
                              )
    #run each of your jobs
    result = job.result(decoder=RunnerResult)

    noisy = result.get_counts()
    mitigated = result.get_quasiprobabilities().nearest_probability_distribution()
    #sim_counts.append(mitigated_results.get_counts())
    
    for state in ['000','001','010','011','100','101','110','111']:
                check = state in sim_counts_mitigated.keys()
                if check == False:
                    sim_counts_mitigated.update({state : 0})
                sim_counts_mitigated[state] = int(sim_counts_mitigated[state])
                print(sim_counts_mitigated[state])
    
    sim_counts.append(sim_counts_mitigated)
    #result= job.result()
    #counts.append(result.get_counts())
  
    #print(result.get_counts()) 

    #Compute the means and standard deviations and store them into the correct variables    
    sim_means.append(get_mean_fermion_number(sim_counts[T])) 
    noisy_means.append(get_mean_fermion_number(noisy_counts[T]))
    #means.append(get_mean_fermion_number(counts[T]))  
    
    sim_errs.append(get_bootstrap_error(sim_counts[T]))
    errs.append(get_bootstrap_error(noisy_counts[T]))
    
    #print out the time elapsed,means, and errors
    #print(T*epsilon, sim_means[T], sim_errs[T], means[T], errs[T])
    
    #print(T*epsilon, sim_means[T], sim_errs[T])
    print(T*epsilon, sim_means[T])
    print('noise')
    print(T*epsilon, noisy_means[T])


